In [1]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as ptc
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from scipy.stats import entropy, wasserstein_distance, truncnorm, pearson3, entropy
import emcee
from tqdm import tqdm
import scipy.stats as sp
from matplotlib import cm
from matplotlib.colors import Normalize
mpl.rcParams['figure.dpi'] = 500

In [ ]:
import os
import numpy as np
from scipy.stats import pearson3
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
mediandiameter = 1
def hkm(parm):
    mean_log = parm[0]
    std_log = parm[1]
    skew = parm[2]
    rvs_pearson3 = pearson3.rvs(skew=skew, loc=mean_log, scale=std_log, size=100)
    input_shear_stress_array = np.exp(rvs_pearson3)
    total_sediments = 100000
    diameters = np.random.uniform(0.01, 2.00, total_sediments)
    shield = 0.05
    specificg = 2
    scaled_r_values = [shield * (specificg - 1) * 1000 * 9.8 * r for r in diameters]
    x_coordinates = np.random.uniform(-1000, 0, total_sediments)
    y_coordinates = np.random.uniform(0, 10, total_sediments)
    x_coordinates_all_iterations = []
    y_coordinates_all_iterations = []

    for iteration, input_shear_stress in tqdm(enumerate(input_shear_stress_array), total=len(input_shear_stress_array)):
        move_indices = np.where(np.array(scaled_r_values) < input_shear_stress)[0]
        move_diameters = np.array(diameters)[move_indices]
        diameterratio = move_diameters/mediandiameter
        L_values = 1.77 * (1 - np.log10(diameterratio))**1.35
        Distance = L_values
        x_coordinates[move_indices] += Distance
        x_coordinates_all_iterations.append(x_coordinates.copy())
        y_coordinates_all_iterations.append(y_coordinates.copy())

    return input_shear_stress_array, x_coordinates_all_iterations, y_coordinates_all_iterations, diameters

input_shear_stress_array, x_coordinates_all_iterations, y_coordinates_all_iterations, diameters = hkm([2, 2, 2])

os.chdir('/Users/braydennoh/Downloads/StatSed')
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
norm = Normalize(vmin=input_shear_stress_array.min(), vmax=input_shear_stress_array.max())
ax1.plot(input_shear_stress_array, color='blue')
marker, = ax1.plot([], [], 'ro')
ax1.set_title('Input Shear Stress Array')
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.set_title('Particle Transport')
scat = ax2.scatter([], [])

def init():
    marker.set_data([], [])
    scat.set_offsets(np.empty((0, 2)))  # Set an empty array with 2 columns
    return marker, scat,

def update(iteration):
    marker.set_data(iteration, input_shear_stress_array[iteration])
    x_coords = x_coordinates_all_iterations[iteration]
    y_coords = y_coordinates_all_iterations[iteration]

    # You can modify the size calculation as per your requirement
    sizes = np.array(diameters) * 50  # Multiplied by 50 for better visibility
    
    # Assuming you want to color particles by their size
    norm_sizes = Normalize(vmin=sizes.min(), vmax=sizes.max())
    
    scat = ax2.scatter(x_coords, y_coords, s=sizes, c=sizes, cmap=cm.Blues, norm=norm_sizes)
    return marker, scat,

ani = FuncAnimation(fig, update, frames=len(input_shear_stress_array), init_func=init, blit=True)
ani.save('particle_transport.gif', writer='imagemagick', fps=10)

if not os.path.exists('testimage'):
    os.makedirs('testimage')

# Iterate through frames and save images
for iteration in tqdm(range(len(input_shear_stress_array))):
    update(iteration)
    fig.savefig(f'testimage/frame_{iteration:04d}.png')


100%|██████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 336.42it/s]
MovieWriter imagemagick unavailable; using Pillow instead.
/var/folders/pv/vxz_z6x905j13byjd_vxl_br0000gn/T/ipykernel_71971/2340847332.py:55: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  marker.set_data(iteration, input_shear_stress_array[iteration])
